Apply the previously created model to real-time data to play the game.

In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function
import numpy as np
import cv2
from PIL import ImageGrab
import pyautogui

import matplotlib.pyplot as plt
plt.style.use('ggplot')

from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import Adam
from keras import activations
from keras.preprocessing.image import ImageDataGenerator

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import time
import sys

In [3]:
#utility functions

#for driver
driver = ""
chrome_path = r"chromedriver_win32\chromedriver.exe" 
startpage = "chrome://dino/"

def initDriver(): #set up driver and connect  
    global driver
    chrome_options = Options()
    chrome_options.add_argument("--window-size=640,480")
    driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
    driver.get(startpage)

def endDriver(): #close driver
    global driver
    driver.quit()
    
def processImage(frame): 
    greyImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, BWImg = cv2.threshold(greyImg,127,255,cv2.THRESH_BINARY)
    processedImg = cv2.resize(BWImg,(120, 40), interpolation = cv2.INTER_AREA)
    return processedImg
 

In [4]:
#load model
model = load_model('DinoRunner_AI_Model.h5')

In [17]:
#initialize driver

initDriver()
pyautogui.FAILSAFE = True

#delayed start
for i in list(range(3))[::-1]:
    print(i+1)
    time.sleep(1)
    
cv2.namedWindow('output', cv2.WINDOW_NORMAL)
cv2.resizeWindow('output', 600, 220)
    
#select game window
pyautogui.click(x=100, y=400)
                
#start the game
#driver.find_element_by_tag_name('body').send_keys(' ')
pyautogui.press('up')

last_time = time.time()
times = []
key = 0
start_time = time.time()
while(True):
    #screen capture
    frame = np.array(ImageGrab.grab(bbox=(30,210, 630, 410))) #score cut
    
    times.append(time.time()-last_time)
    last_time = time.time()
    current_time = int(round(time.time() - start_time)) + 15
    
    #process capture
    frame_processed = processImage(frame)
    
    #add time in seconds so AI knows how fast to go
    cv2.putText(frame_processed,
                text = str(current_time-15),
                org = (60,10),
                fontFace = cv2.FONT_HERSHEY_PLAIN,
                fontScale = 1,
                color = 127)
    
    frame_tensor = np.array(frame_processed).astype('float32')/255
    
    #need appropriate tensor: convert to shape (1, 40, 120, 1)
    frame_tensor = np.expand_dims(frame_tensor, axis = 0)
    frame_tensor = np.expand_dims(frame_tensor, axis = 3)
    
    #input to model
    key = model.predict(frame_tensor)
    
    if (key > 0.20):
        pyautogui.press('up')
    
    #put key on screen so we can see its decision
    cv2.putText(frame_processed,
                text = "{0:.2f}".format(float(key)),
                org = (20,10),
                fontFace = cv2.FONT_HERSHEY_PLAIN,
                fontScale = 0.7,
                color = 127)
    
    key = 0 #reset for next frame
    
    #output of capture
    cv2.imshow('output', frame_processed) #processed
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        print('Average frame time: {} seconds'.format(np.mean(times)))
        print('Average Framerate: {} fps'.format(1/np.mean(times)))
        #CPU single-threaded limited performance (numpy, opencv)
        endDriver()
        break

3
2
1
Average frame time: 0.12017183155976524 seconds
Average Framerate: 8.32141764854993 fps
